# Pipeline 3: Categorical patient demographics for predicting the length of stay

This notebook presents all code used for the evaluation of the code produced for the third pipeline, as well as the related relational algebra

# Relational algebra
$D_{inputevents}(subject\_id, hadm\_id, stay\_id, caregiver\_id, starttime, endtime, storetime, itemid, amount, amountuom, rate, rateuom, orderid, linkorderid, ordercategoryname, secondaryordercategoryname, ordercomponenttypedescription, ordercategorydescription, patientweight, totalamount, totalamountuom, isopenbag, continueinnextdept, statusdescription, originalamount, highnormalvalue)$

$D_{d\_items}(itemid, label, abbreviation, linksto, category, unitname, param\_type, lownormalvalue, conceptid)$

$inputevents\_rate = \sigma_{rate IS NOT NULL}(inputevents)$

$d\_items\_s = \sigma_{*}(d\_items)$
 
$inputevents\_d\_items = inputevents\_rate\bowtie_{inputevents\_rate.itemid = d\_items\_s.itemid}d\_items\_s$

$inputevents\_d\_items = \rho_{patient\_id/subject\_id}(inputevents\_d\_items)$

$final\_data = \pi_{subject\_id, hadm\_id, starttime, endtime, itemid, label, amount, rate, rateuom and totalamount}(inputevents\_d\_items)$

$final\_data  = \tau_{starttime}(final\_data)$

In [ ]:
# Necessary imports and setup of connection to the database

import polars as pl
from sqlalchemy import create_engine, inspect, Table, MetaData, Column, Integer, String, DateTime, Float, Boolean
import numpy as np

engine = create_engine(r"sqlite://path to where mimic4.db is stored") #change this to the path where mimic.db is stored. If timeout occurs, add pool_size and pool_timeout to the engine creation

In [ ]:
# Ground truth code, used for comparison
metadata = MetaData()

inputevents = Table(
    'inputevents', metadata,
    Column('subject_id', Integer, nullable=False),
    Column('hadm_id', Integer, nullable=False),
    Column('stay_id', Integer, primary_key=True),
    Column('caregiver_id', Integer, nullable=False),
    Column('starttime', DateTime, nullable=False),
    Column('endtime', DateTime, nullable=True),
    Column('storetime', DateTime, nullable=False),
    Column('itemid', Integer, ForeignKey("d_items.itemid"), nullable=False),
    Column('amount', Float, nullable=False),
    Column('amountuom', String, nullable=True),
    Column('rate', Float, nullable=True),
    Column('rateuom', String, nullable=True),
    Column('orderid', Integer, nullable=False),
    Column('linkorderid', Integer, nullable=False),
    Column('ordercategoryname', String, nullable=False),
    Column('secondaryordercategoryname', String, nullable=True),
    Column('ordercomponenttypedescription', String, nullable=True),
    Column('ordercategorydescription', String, nullable=False),
    Column('patientweight', Float, nullable=False),
    Column('totalamount', Float, nullable=False),
    Column('totalamountuom', String, nullable=False),
    Column('isopenbag', Boolean, nullable=False),
    Column('continueinnextdept', Integer, nullable=False),
    Column('statusdescription', String, nullable=False),
    Column('originalamount', Float, nullable=True),
    Column('highnormalvalue', Float, nullable=True)
)


d_items = Table(
    'd_items', metadata,
    Column('itemid', Integer, primary_key=True, nullable=False),
    Column('label', String, nullable=False),
    Column('abbreviation', String, nullable=False),
    Column('linksto', String, nullable=False),
    Column('category', String, nullable=False),
    Column('unitname', String, nullable=True),
    Column('param_type', String, nullable=False),
    Column('lownormalvalue', String, nullable=True),
    Column('conceptid', Integer, nullable=True)
)


metadata.reflect(bind=engine)

query = """
SELECT *
FROM inputevents
WHERE rate is not null
"""

query2 = """
SELECT *
FROM d_items
"""

df1 = pl.read_database(query=query, connection=engine.connect())
df2 = pl.read_database(query=query2, connection=engine.connect(), infer_schema_length=10000)


df1_df2 = df1.join(df2, on="itemid")

df1_df2 = df1_df2.rename({"subject_id": "patient_id"})

df1_df2 = df1_df2.select(['patient_id', 'hadm_id', 'starttime', 'endtime', 'itemid', 'label', 'amount', 'rate', 'rateuom', 'totalamount'])

df1_df2 = df1_df2.sort('starttime')

df1_df2

# Code Valdity 

In the code block below, the generated code produced by the LLM can be pasted and executed, to assess if the code works without any runtime errors.

In [ ]:
# Paste generated code here 

# Code correctness

To assess the code correctness, the code below can be executed. If it is correct, True should be returned. Otherwise, manual inspection in the code block below should be executed. It is assumed the presented solution is stored in a variable called 'final data'.

In [ ]:
df1_df2.equals(final_data)

In [ ]:
#  Execute this code if the dataframes are not equal to see the differences
df_combined_pandas = df_combined.to_pandas()
final_data_pandas = final_data.to_pandas()

df_diff = df_combined_pandas.compare(final_data_pandas)

if not df_combined_pandas.equals(final_data_pandas):
    print(df_diff)

# Relative Efficiency

To record the average running time for the generated solution, the generated solution can be added to the code block below and executed. The average executrion time should be recorded per solution, for later determination of the relative efficiency

In [ ]:
import time
import numpy as np
import warnings

# Ignore all warnings
warnings.filterwarnings('ignore')

execution_times = []

for i in range(10):
    start_time = time.time()

    # Paste generated code here

    end_time = time.time()

    execution_time = end_time - start_time
    execution_times.append(execution_time)


average_execution_time = np.mean(execution_times)

print(f"The code executed in average {average_execution_time} seconds over 10 runs.")

In [ ]:
# To calculate the relative efficiency of the generated solutions
def relative_efficiencies(times):
    tmin = min(times)
    tmax = max(times)
    efficiencies = [100 * (1 - (tc - tmin) / (tmax - tmin)) for tc in times]
    return efficiencies

time_gt = [204.8] # ground truth time
solutions = [] # Add the times from the 10 runs here
times = time_gt + solutions


re = relative_efficiencies(times)
# Calculate average
average = np.mean(re[1:])
print(f'Average: {average}')